In [3]:
import cv2

hand_cascade_path = 'D:/Git/hand_detect/data/fist.xml'
user_choice_for_movement_detector_type = 1  # Change this to 2 for palm detection

# Load the hand cascade classifier
opencv = cv2.CascadeClassifier(hand_cascade_path)

# Function to perform hand detection
def detect_hands(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hands = opencv.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return hands

# Capture video from the default camera
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Failed to open camera.")
    exit()

# Loop to capture and process video frames
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    #get the center position of the window
    window_height, window_width, _ = frame.shape
    center_x = window_width/2
    center_y = window_height/2

    # Perform hand detection
    hands = detect_hands(frame)

    # Avoid too much hand on the camera
    n = len(hands)
    max = 0
    def cal_dis(x,y):
        return (abs(x-center_x)**2)+(abs(y-center_y)**2)
    
    # last x and y
    x_new = 0
    y_new = 0 
    w_x = 0 
    h_y = 0
    # Eliminating noise point (taking the fartherst point to the center)
    for (x, y, w, h) in hands:
        if(cal_dis(x,y) > max):
            x_new = x 
            y_new = y
            w_x = w
            h_y = h
    
    #Display the last point after eliminating noise point
    cv2.rectangle(frame, (x_new, y_new), (x_new + w_x, y_new + h_y), (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Hand Detection', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


In [1]:
import numpy as np
import face_recognition
import torch
from torchvision import transforms
from model import Model 
import cv2
model = Model()
model.load_state_dict(torch.load('D:/FER/train_testresnet50eac/2_model_final_weights.pth', map_location= 'cpu'))
device = torch.device('cpu')

eval_transform  = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean = [0.485, 0.456, 0.406],
                         std = [0.229, 0.224, 0.225]),
])
emotion  = ['suprise', 'fear', 'disgust', 'happy', 'sad', 'angry', 'neutral']


cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Không thể mở camera. Kiểm tra lại index hoặc camera có được kết nối không.")
    exit()
        # Đợi cho đến khi có frame từ camera
while True:
    ret, frame = cap.read()
    if not ret:
        print("Không thể nhận frame từ camera.")
        break
    
    # Hiển thị frame
    image = np.array(frame)
    face_locations = face_recognition.face_locations(image)
    
    face_image = image
    for face_location in face_locations:
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, bottom), (right, top), (0, 255, 0), 2)
        temp  = image[top:bottom, left:right]
        face_image = temp
    
    
    face_image = face_image[:, :, ::-1]
    face_image = eval_transform(face_image)
    face_image= torch.unsqueeze(face_image, dim = 0)
    with torch.no_grad():
        model.eval()
        face_image = face_image.to(device)
        outputs, _ = model(face_image)
        _, predicts = torch.max(outputs, 1)
        x = emotion[predicts]

    cv2.imshow('camera', frame)
            # Chờ ấn phím 'q' để chụp ảnh và thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'model'